In [1]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

2023-08-07 19:00:38 GM01X680 metapub.config[143322] WARNING NCBI_API_KEY was not set.


In [2]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
# www.nature.com
def www_nature_com(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            time.sleep(5)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    try:
        doi = driver.find_element(By.XPATH, "//li[@class='c-bibliographic-information__list-item c-bibliographic-information__list-item--doi']").find_element(By.TAG_NAME, "p").find_element(By.XPATH, "//span[@class='c-bibliographic-information__value']").text
    except:
        doi = np.nan
    pmid = np.nan
    pmcid = np.nan
    try:
        title = driver.find_element(By.XPATH, "//h1[contains(@class, 'c-article-title')]").text
    except:
        title = np.nan
    abstract = np.nan
    keywords = np.nan
    intro = np.nan
    pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }
    driver.quit

    return info
# --------------------start of test code--------------------
url = "https://www.nature.com/articles/387281a0"
info = www_nature_com(url)
print(info["doi"])
print(info["pmid"])
print(info["pmcid"])
print(info["title"])
print(info["abstract"])
print(info["keywords"])
print(info["introduction"])
print(info["pdf_link"])
# ---------------------end of test code---------------------

04 December 1996
nan
nan
Dynamics of orientation tuning in macaque primary visual cortex
nan
nan
nan
nan
